# Week 6 Exercise

# Fire up GraphLab Create

In [10]:
import graphlab
graphlab.canvas.set_target('ipynb')

# Load a common image analysis dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set.

In [3]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1495426981.log


This non-commercial license of GraphLab Create for academic use is assigned to onlydevad@gmail.com and will expire on March 27, 2018.


# Solution 1

In [4]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


# Solution 2

In [5]:
cat_model = graphlab.nearest_neighbors.create(image_train[image_train['label'] == 'cat'],
                                             features=['deep_features'], label = 'id')

Starting brute force nearest neighbors model training.

In [6]:
dog_model = graphlab.nearest_neighbors.create(image_train[image_train['label'] == 'dog'],
                                             features=['deep_features'], label = 'id')

Starting brute force nearest neighbors model training.

In [7]:
bird_model = graphlab.nearest_neighbors.create(image_train[image_train['label'] == 'bird'],
                                             features=['deep_features'], label = 'id')

Starting brute force nearest neighbors model training.

In [8]:
automobile_model = graphlab.nearest_neighbors.create(image_train[image_train['label'] == 'automobile'],
                                             features=['deep_features'], label = 'id')

Starting brute force nearest neighbors model training.

In [19]:
cat = image_test[0:1]
print "Nearest ‘dog’ and ‘cat’ labeled image in the training data to the cat image above -->"
print "ID of nearest ‘cat’ labeled image : %s" % cat_model.query(car, verbose=0, k=1)['reference_label'][0]
print "ID of nearest ‘dog’ labeled image : %s" % cat_model.query(car, verbose=0, k=1)['reference_label'][0]

Nearest ‘dog’ and ‘cat’ labeled image in the training data to the cat image above -->
ID of nearest ‘cat’ labeled image : 16289
ID of nearest ‘dog’ labeled image : 16289


# Solution 3

In [27]:
cat = image_test[0:1]
print "Mean distance between cat image at its 5 nearest neighbors that were labeled ‘cat’ and ‘dog’ in the training data"
print "Mean for 'cat' labeled data : %s" % cat_model.query(cat, verbose=0)['distance'].mean()
print "Mean for 'dog' labeled data : %s" % dog_model.query(cat, verbose=0)['distance'].mean()

Mean distance between cat image at its 5 nearest neighbors that were labeled ‘cat’ and ‘dog’ in the training data
Mean for 'cat' labeled data : 36.1557307098
Mean for 'dog' labeled data : 37.7707113618


# Solution 4

In [68]:
dog_distances = graphlab.SFrame({
        'dog-dog' : dog_model.query(image_test[image_test['label'] == 'dog'], verbose=0, k=1)['distance'],
        'dog-cat' : cat_model.query(image_test[image_test['label'] == 'dog'], verbose=0, k=1)['distance'],
        'dog-bird' : bird_model.query(image_test[image_test['label'] == 'dog'], verbose=0, k=1)['distance'],
        'dog-automobile' : automobile_model.query(image_test[image_test['label'] == 'dog'], verbose=0, k=1)['distance']
    })

In [71]:
dog_distances.apply(lambda row: 1 if row['dog-dog'] == min(row.values()) else 0).sum()/float(len(dog_distances))

0.678